# PyTorch 基本运算示例

Torch的tensor和Numpy的Array，Pandas的Series和Dataframe类似，都是了解它们的基础，所以这里日常积累一些常用张量运算函数。

主要参考：

- [TENSORS](https://pytorch.org/tutorials/beginner/former_torchies/tensor_tutorial.html)。

In [10]:
import numpy as np
import torch

X = np.random.rand(6,4)
print(X,X.shape)
avg_np, _ = np.average(X, axis=0, returned=True)
avg_np, _

[[0.85832951 0.17197487 0.84933902 0.78488964]
 [0.70293221 0.88172093 0.82549544 0.73390744]
 [0.06626581 0.75575926 0.03123777 0.4620981 ]
 [0.76854514 0.92926573 0.34542583 0.47172337]
 [0.46905619 0.00569534 0.36139286 0.68522161]
 [0.4085438  0.46559733 0.53468566 0.25821807]] (6, 4)


(array([0.54561211, 0.53500224, 0.49126277, 0.56600971]),
 array([6., 6., 6., 6.]))

In [11]:
X_th = torch.tensor(X)
avg_th = torch.mean(X_th, dim=0)
avg_th

tensor([0.5456, 0.5350, 0.4913, 0.5660], dtype=torch.float64)

In [13]:
assert (X == X_th.numpy()).all()
# assert (avg_np == avg_th.numpy()).all()   # this fails alread

求和运算和平均运算类似：

In [30]:
ta=torch.ones(5)
tc=torch.sum(ta)
tc

tensor(5.)

类似numpy的repeat和tile，torch中可以使用repeat：

In [15]:
z = torch.FloatTensor([[1,2,3],[4,5,6]])
z.repeat(1,4)

tensor([[1., 2., 3., 1., 2., 3., 1., 2., 3., 1., 2., 3.],
        [4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.]])

In [16]:
z.repeat(1,4).view(-1, 3)

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.]])

In [17]:
z.repeat(4,1)

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [1., 2., 3.],
        [4., 5., 6.],
        [1., 2., 3.],
        [4., 5., 6.],
        [1., 2., 3.],
        [4., 5., 6.]])

初始化张量1：

In [21]:
ta=torch.ones(5)

tensor的element-wise运算。

In [22]:
tb=torch.zeros(5)
ta-tb

tensor([1., 1., 1., 1., 1.])

In [27]:
z = torch.FloatTensor([2]).repeat(1,5)
z

tensor([[2., 2., 2., 2., 2.]])

In [28]:
z/ta

tensor([[2., 2., 2., 2., 2.]])

In [31]:
td=ta/z
td

tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000]])

In [32]:
te=td**2
te

tensor([[0.2500, 0.2500, 0.2500, 0.2500, 0.2500]])

看看二维的情况：

In [35]:
t1 = torch.FloatTensor([[1,2,3],[4,5,6]])
t2=t1.repeat(1,4).view(-1, 3)
t2

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.]])

In [36]:
t3 = torch.FloatTensor([[7,8,9],[10,11,12]])
t4=t3.repeat(1,4).view(-1, 3)
t4

tensor([[ 7.,  8.,  9.],
        [ 7.,  8.,  9.],
        [ 7.,  8.,  9.],
        [ 7.,  8.,  9.],
        [10., 11., 12.],
        [10., 11., 12.],
        [10., 11., 12.],
        [10., 11., 12.]])

In [45]:
SST = (t4 - t2) ** 2
SST

tensor([[36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.]])

In [46]:
SST=torch.sum(SST,dim=0)
SST

tensor([288., 288., 288.])

有时候会有一nan值需要处理，有一种技巧如下所示，可以利用这个作为判断一个tensor有没有nan值得方法。

In [41]:
print(np.nan==np.nan)

False


In [42]:
t5 = torch.FloatTensor([[7,np.nan,9],[10,11,12]])
t5

tensor([[ 7., nan,  9.],
        [10., 11., 12.]])

In [43]:
mask= t5==t5
mask

tensor([[ True, False,  True],
        [ True,  True,  True]])

In [44]:
t6=t5[mask]
t6

tensor([ 7.,  9., 10., 11., 12.])

In [48]:
def nse_2d(t,p):
    seq_length = t.shape[0]
    Ngage = t.shape[1]
    tmean = torch.mean(t, dim=0)
    tmeans = tmean.repeat(seq_length, 1)
    SST = torch.sum((t - tmeans) ** 2, dim=0)
    SSRes = torch.sum((t - p) ** 2, dim=0)
    # Same as Fredrick 2019
    # temp = SSRes / ((torch.sqrt(SST) + 0.1) ** 2)
    # original NSE
    temp = SSRes / SST
    loss = torch.sum(temp) / Ngage
    return loss

t1 = torch.FloatTensor([[1,2,3],[4,5,6]])
t2 = torch.FloatTensor([[1,2,3],[4,5,6]])
print(nse_2d(t1,t2))

tensor(0.)


试试torch的广播功能：

In [49]:
t1 = torch.FloatTensor([[1,2,3],[4,5,6]])
t2=torch.sqrt(t1)

tensor([[1.0000, 1.4142, 1.7321],
        [2.0000, 2.2361, 2.4495]])

In [52]:
t1/((t2+0.1)**2)

tensor([[0.8264, 0.4535, 0.3122],
        [0.2380, 0.1922, 0.1612]])

tensor 转换为numpy：

In [1]:
import torch
a = torch.ones(5)
print(a)

tensor([1., 1., 1., 1., 1.])


In [2]:
b = a.numpy()
print(b)

[1. 1. 1. 1. 1.]


In [3]:
a.add_(1)
print(a)
print(b)    # see how the numpy array changed in value

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


注意tensor变化时，array也会跟着变。反过来也一样：

In [4]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)  # see how changing the np array changed the torch Tensor automatically

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


All the Tensors on the CPU except a CharTensor support converting to NumPy and back.

再看看维度变换，三维变二维：

In [2]:
import torch
t1 = torch.FloatTensor([[[7,8,9],[10,11,12]],[[7,8,9],[10,11,12]]])
t1

tensor([[[ 7.,  8.,  9.],
         [10., 11., 12.]],

        [[ 7.,  8.,  9.],
         [10., 11., 12.]]])

In [4]:
t2=t1.view(-1, 3)
t2

tensor([[ 7.,  8.,  9.],
        [10., 11., 12.],
        [ 7.,  8.,  9.],
        [10., 11., 12.]])

In [5]:
t3=t1.reshape(-1, 3)
t3

tensor([[ 7.,  8.,  9.],
        [10., 11., 12.],
        [ 7.,  8.,  9.],
        [10., 11., 12.]])

reshape 和 view 的区别这里简单补充下，参考了：[Pytorch-reshape与view的区别](https://congluwen.top/2018/12/pytorch_reshape_view/)。Pytorch中reshape()与view()都可以改变Tensor的shape但是有略微的区别.

- view()只可以由torch.Tensor.view()来调用，view():
    - 不改变Tensor数据，改变Tensor的size(即shape)
    - 对于一个将要被view的Tensor，新的size必须与原来的size与stride兼容,即新的维度必须是以下两种情况: 
        - 是原有维度的一个子空间
        - 只跨越原有满足邻接条件，stride[i]=stride[i+1]×size[i+1]的原有维度d,d+1,…,d+k\
    - 否则，在view之前必须调用contiguous()方法，对于该方法的讨论，见[StackOverflow](https://stackoverflow.com/questions/48915810/pytorch-contiguous)
- reshape()可以由torch.reshape(),也可由torch.Tensor.reshape()调用。reshape():
    - 同样也是返回与input数据量相同，但形状不同的tensor
    - 若满足view的条件，则不会copy，若不满足，则会copy

张量的转置：

In [6]:
import torch
x = torch.randn(2, 3)
x

tensor([[-0.0589, -0.2256, -1.2757],
        [-0.5401,  1.4478, -1.2537]])

In [7]:
torch.t(x)

tensor([[-0.0589, -0.5401],
        [-0.2256,  1.4478],
        [-1.2757, -1.2537]])

关于torch中的nan值，在torch中检测是否有nan值可以使用：x != x

In [9]:
import torch
import numpy as np
x = torch.tensor([1, 2, np.nan])
x != x

tensor([False, False,  True])

如果是直接判断一个多维张量是否有nan值，可以使用：

In [10]:
mask = x != x
if len(mask[mask == True]) > 0:
    print("please check")

please check


复制张量：

In [1]:
import torch
x = torch.randn(2, 3, 4)
x

tensor([[[ 0.3281, -0.8897,  0.7370, -0.9295],
         [ 0.7451,  0.3618, -0.4559, -0.7497],
         [-0.9598, -1.1446, -2.7180, -1.1157]],

        [[ 1.1102, -1.1062, -0.7549,  1.4117],
         [ 0.5135, -1.6110,  0.4967,  1.1779],
         [ 0.5094,  1.6470,  0.0407,  0.1715]]])

In [2]:
x[-1,:,:]

tensor([[ 1.1102, -1.1062, -0.7549,  1.4117],
        [ 0.5135, -1.6110,  0.4967,  1.1779],
        [ 0.5094,  1.6470,  0.0407,  0.1715]])

In [3]:
x[-1,:,:].repeat(3,1,1)

tensor([[[ 1.1102, -1.1062, -0.7549,  1.4117],
         [ 0.5135, -1.6110,  0.4967,  1.1779],
         [ 0.5094,  1.6470,  0.0407,  0.1715]],

        [[ 1.1102, -1.1062, -0.7549,  1.4117],
         [ 0.5135, -1.6110,  0.4967,  1.1779],
         [ 0.5094,  1.6470,  0.0407,  0.1715]],

        [[ 1.1102, -1.1062, -0.7549,  1.4117],
         [ 0.5135, -1.6110,  0.4967,  1.1779],
         [ 0.5094,  1.6470,  0.0407,  0.1715]]])